In [1]:
!git clone https://github.com/EmperorNao/wda.git
!pip install -r wda/requirements.txt

fatal: destination path 'wda' already exists and is not an empty directory.


In [2]:
%run -i 'wda/dataset.py'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import codecs
from bs4 import BeautifulSoup
import time



LOCAL = "/content"
dataset = Dataset(
    path_to_groups={
        'test': join(LOCAL, "test_groups.csv"),
        'train': join(LOCAL, "train_groups.csv"),
    },
)

In [4]:
# dataset.from_csv("simple_data.csv")

In [5]:
dataset.from_csv(join("/content", 'extended_dataset.csv'))

In [6]:
dataset[:]

,Unnamed: 0,pair_id,group_id,doc_id,target,IsTrain,title,keywords,abstract,description,title_len
0,0,1,1,15731,0.0,1,ваз 21213 | замена подшипник ступица | нива,"замена подшипник ступица , технический описани...",NaN,"замена подшипник ступица , руководство ремонт ...",8
1,1,2,1,14829,0.0,1,"ваз 2107 оптом сочи . сравнить цена , купить п...","ваз 2107 оптом , россия , сочи , цена , предло...","ваз 2107 опт . продажа , поиск , поставщик маг...","ваз 2107 опт . продажа , поиск , поставщик маг...",14
2,2,3,1,15764,0.0,1,купить ступица лада калина 2 . трансмиссия - п...,"цена , замена , тюнинг , купить , продать , ст...",NaN,продажа запчасть ступица легковой грузовой авт...,16
3,3,4,1,17669,0.0,1,классика 21010 - 21074,NaN,NaN,NaN,4
4,4,5,1,14852,0.0,1,ступица нива — замена подшипник свой рука,NaN,NaN,передний ступица нива иметь свой особенность ....,7
...,...,...,...,...,...,...,...,...,...,...,...
28312,28312,28313,309,16637,NaN,0,ответ @ mail . ru : полезный кушать творог утр...,NaN,NaN,пользователь лили гейбиф задать вопрос категор...,18
28313,28313,28314,309,16759,NaN,0,творог . полезный свойство лечение творог . же...,творог лечебный полезный целебный свойство тво...,NaN,"творог обладать многий полезный , лечебный ( ц...",14
28314,28314,28315,309,15358,NaN,0,творог - полезный опасный свойство творог,творог свойство,NaN,"творог , химический состав , пищевой ценность ...",6
28315,28315,28316,309,17287,NaN,0,ответ @ mail . ru : полезный творог ?,NaN,NaN,пользователь кит кит задать вопрос категория п...,9


In [7]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

In [8]:
#!pip install umap-learn


In [9]:

# import pandas as pd
# import numpy as np
# import plotly.express as px

# from sklearn import model_selection, ensemble, metrics, cluster
# import umap
# from functools import partial

# import torch
# from transformers import AutoTokenizer, AutoModel

# def embed_bert_cls(text, model, tokenizer):
#     t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

#     with torch.no_grad():
#         model_output = model(**{k: v.to(model.device) for k, v in t.items()})

#     embeddings = model_output.last_hidden_state[:, 0, :]
#     embeddings = torch.nn.functional.normalize(embeddings)
#     return embeddings[0].cpu().numpy()



# def rubert_embeddings(s: pd.Series) -> np.ndarray:
#     return np.vstack(s.map(partial(encode_sents, model=model, tokenizer=tokenizer)).values)


# def umap_embeddings(X: np.ndarray):
#     return umap.UMAP(
#         random_state=100,
#         n_neighbors=10,
#         n_components=3,
#         n_jobs=1,
#         metric="cosine",
#         min_dist=0.1,
#     ).fit_transform(X)


# def optics_clusterize(X: np.ndarray) -> np.ndarray:
#     return cluster.OPTICS(metric="euclidean").fit(X).labels_

In [10]:
# import pandas as pd
# import numpy as np
# import plotly.express as px

# from sklearn import model_selection, ensemble, metrics, cluster
# import umap

In [11]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

In [12]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, classification_report

In [13]:
def measure_std(sim, first_sents):
    first_sim = sim[first_sents,:][:,first_sents]
    np.fill_diagonal(first_sim, 0)
    return first_sim.std(axis=0).mean()

In [14]:

import pandas as pd
title_data = dataset[:].iloc[:, 1:]#pd.read_csv('docs_titles.tsv', sep='\t', encoding='utf-8', lineterminator='\n')
# title_data[0]['title']
title_data.loc[0]['title']

train_groups = pd.read_csv('train_groups.csv')
print (train_groups.shape)
train_groups.head()

test_groups = pd.read_csv('test_groups.csv')
print (test_groups.shape)
test_groups.head()


(11690, 4)
(16627, 3)


,pair_id,group_id,doc_id
0,11691,130,6710
1,11692,130,4030
2,11693,130,5561
3,11694,130,4055
4,11695,130,4247


In [15]:
# title_data = title_data.fillna('')

In [16]:
doc_to_title = title_data

In [17]:
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in tqdm(range(len(train_data))):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title.loc[doc_to_title['doc_id'] == doc_id, 'title'].iloc[0]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

100%|██████████| 11690/11690 [00:10<00:00, 1157.92it/s]


In [18]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in tqdm(range(len(test_data))):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    pair_id = new_doc['pair_id']
    title = doc_to_title.loc[doc_to_title['doc_id'] == doc_id, 'title'].iloc[0]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title, pair_id))

100%|██████████| 16627/16627 [00:11<00:00, 1435.47it/s]


In [1]:
import numpy as np
X_train_most_words = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train_most_words.append(sorted(all_dist, reverse=True)[0:15]    )
X_train_most_words = np.array(X_train_most_words)
groups_train = np.array(groups_train)


X_test_most_words = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test_most_words.append(sorted(all_dist, reverse=True)[0:15]    )
X_test_most_words = np.array(X_test_most_words)
groups_test = np.array(groups_test)

NameError: ignored

In [20]:
PUNKT = "|[]{},./\\\"\'`~"

top_k = 15

X_train_top_words = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]

    all_words = []
    words = set()
    for k, (doc_id, title, target_id) in enumerate(docs):
        cur_words = ' '.join(filter(lambda x: x not in PUNKT, title.strip().split()))
        all_words.append(cur_words)
        words = words & set(cur_words)

    cv = np.array(CountVectorizer().fit_transform(all_words).todense())
    counts = cv.sum(axis=0)

    idxs = np.argsort(counts)[::-1]
    ret = np.zeros((cv.shape[0], top_k))
    ret[:,:min(idxs.shape[0], top_k)] = cv[:, idxs[:min(idxs.shape[0], top_k)]]

    one_idxs = np.where(np.logical_and(counts >= 3, counts <= 10))[0]

    n_less_word = cv[:, one_idxs].sum(axis=1)[:, None]
    X_train_top_words.append(np.concatenate([ret, n_less_word], axis=1))

X_train_top_words = np.concatenate(X_train_top_words, axis=0)



X_test_top_words = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]

    all_words = []
    for k, (doc_id, title, target_id) in enumerate(docs):
        all_words.append(' '.join(filter(lambda x: x not in PUNKT, title.strip().split())))

    cv = np.array(CountVectorizer().fit_transform(all_words).todense())
    counts = cv.sum(axis=0)
    idxs = np.argsort(counts)[::-1]
    ret = np.zeros((cv.shape[0], top_k))
    ret[:,:min(idxs.shape[0], top_k)] = cv[:, idxs[:min(idxs.shape[0], top_k)]]

    one_idxs = np.where(np.logical_and(counts >= 3, counts <= 10))[0]

    n_less_word = cv[:, one_idxs].sum(axis=1)[:, None]

    X_test_top_words.append(np.concatenate([ret, n_less_word], axis=1))

X_test_top_words = np.concatenate(X_test_top_words, axis=0)

In [21]:
def encode_sents(sents, model, tokenizer):

    model.to('cuda')
    encoded_input = tokenizer(sents, padding=True, truncation=True, max_length=128, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input.to('cuda'))

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).detach().cpu()

    return sentence_embeddings


def get_avg_sent(group_sents):
    return group_sents.mean(axis=0, keepdim=True)

def get_std_sent(group_sents):
    return group_sents.std(axis=0, keepdim=True)

In [23]:
y = []
X_embeddings = []
X_embeddings_cosine = []
X_embeddings_euclidean = []
X_texts = []
for group, data in traingroups_titledata.items():
    sents = []
    for dataitem in data:
        y.append(dataitem[2])
        sents.append(dataitem[1])

    emb = encode_sents(sents, model, tokenizer)
    avg_emb = get_avg_sent(emb)
    diff_embs = emb - avg_emb
    diff_embs_cosine = cosine_similarity(emb, avg_emb)
    diff_embs_euclidean = euclidean_distances(emb, avg_emb)

    X_texts.extend(sents)
    X_embeddings.extend(diff_embs.numpy())
    X_embeddings_cosine.extend(diff_embs_cosine)
    X_embeddings_euclidean.extend(diff_embs_euclidean)


X_embeddings = np.array(X_embeddings)
X_embeddings_euclidean = np.array(X_embeddings_euclidean)
X_embeddings_cosine = np.array(X_embeddings_cosine)


pairs = []
X_embeddings_test = []
X_embeddings_test_cosine = []
X_embeddings_test_euclidean = []
X_texts_test = []
for group, data in testgroups_titledata.items():

    sents = []
    for dataitem in data:
        pairs.append(dataitem[2])
        sents.append(dataitem[1])

    emb = encode_sents(sents, model, tokenizer)
    avg_emb = get_avg_sent(emb)
    diff_embs = emb - avg_emb

    diff_embs_cosine = cosine_similarity(emb, avg_emb)
    diff_embs_euclidean = euclidean_distances(emb, avg_emb)

    X_texts_test.extend(sents)
    X_embeddings_test.extend(diff_embs.numpy())
    X_embeddings_test_cosine.extend(diff_embs_cosine)
    X_embeddings_test_euclidean.extend(diff_embs_euclidean)

X_embeddings_test = np.array(X_embeddings_test)
X_embeddings_test_cosine = np.array(X_embeddings_test_cosine)
X_embeddings_test_euclidean = np.array(X_embeddings_test_euclidean)

In [24]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [26]:
sents_vectorizer = TfidfVectorizer()
sents_decomp = TruncatedSVD(50)

vectorized = sents_vectorizer.fit_transform(X_texts + X_texts_test)
truncated_sents = sents_decomp.fit_transform(vectorized)

emb_decomp = TruncatedSVD(50)
truncated_embeddings = emb_decomp.fit_transform(np.concatenate([X_embeddings,X_embeddings_test], axis=0))

In [27]:
tfidf_full = sents_vectorizer.transform(X_texts)

In [28]:
k_nearest = 25


X_tfidf_min_dist = []
X_tfidf_cosine_min_dist = []
X_tfidf_diff_avg = []
for group, data in tqdm(traingroups_titledata.items()):
    sents = []
    for dataitem in data:
        sents.append(dataitem[1])


    vectors = sents_vectorizer.transform(sents)
    diff_embs_cosine = cosine_similarity(vectors, vectors)
    diff_embs_euclidean = euclidean_distances(vectors, vectors)

    avg_emb = vectors.mean(axis=0)
    diff_embs = vectors - avg_emb

    X_tfidf_diff_avg.append(diff_embs)


    X_tfidf_min_dist.append(np.sort(diff_embs_euclidean)[:, :k_nearest])
    X_tfidf_cosine_min_dist.append(np.sort(diff_embs_cosine)[:, :k_nearest])


X_tfidf_min_dist = np.concatenate(X_tfidf_min_dist, axis=0)
X_tfidf_cosine_min_dist = np.concatenate(X_tfidf_cosine_min_dist, axis=0)
X_tfidf_diff_avg = np.concatenate(X_tfidf_diff_avg, axis=0)


X_tfidf_min_dist_test = []
X_tfidf_cosine_min_dist_test = []
X_tfidf_diff_avg_test = []
for group, data in tqdm(testgroups_titledata.items()):
    sents = []
    for dataitem in data:
        sents.append(dataitem[1])

    vectors = sents_vectorizer.transform(sents)
    diff_embs_cosine = cosine_similarity(vectors, vectors)
    diff_embs_euclidean = euclidean_distances(vectors, vectors)

    avg_emb = vectors.mean(axis=0)
    diff_embs = vectors - avg_emb

    X_tfidf_diff_avg_test.append(diff_embs)

    X_tfidf_min_dist_test.append(np.sort(diff_embs_euclidean)[:, :k_nearest])
    X_tfidf_cosine_min_dist_test.append(np.sort(diff_embs_cosine)[:, :k_nearest])



X_tfidf_min_dist_test = np.concatenate(X_tfidf_min_dist_test, axis=0)
X_tfidf_cosine_min_dist_test = np.concatenate(X_tfidf_cosine_min_dist_test, axis=0)
X_tfidf_diff_avg_test = np.concatenate(X_tfidf_diff_avg_test, axis=0)

100%|██████████| 180/180 [00:02<00:00, 62.92it/s]


In [ ]:
f1 = sents_decomp.transform(sents_vectorizer.transform(X_texts))
f2 = emb_decomp.transform(X_embeddings)
f3 = X_train_most_words
f4 = X_train_top_words
f5 = X_embeddings_cosine
f6 = X_embeddings_euclidean
#f7 = X_noise_proba_data[:, None]
# f8 = X_embeddings_min_dist
# f9 = X_embeddings_cosine_min_dist
#f = np.concatenate([f1, f2, f3, f4, f5, f6, f7], axis=1)
f10 = X_tfidf_min_dist
f11 = X_tfidf_cosine_min_dist
f12 = X_tfidf_diff_avg
f = np.concatenate([f1, f2, f3, f4, f5, f6, f10, f11, f12], axis=1)
print(f.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(f, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
knn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='cosine')
svc = SVC(class_weight='balanced', probability=True)
ridge = RidgeClassifierCV(class_weight='balanced')

knn.fit(scaler.fit_transform(X_train), y_train)

svc.fit(scaler.transform(X_train), y_train)
ridge.fit(scaler.transform(X_train), y_train)

print("KNN")
y_pred_train = knn.predict(scaler.transform(X_train))
y_pred_val = knn.predict(scaler.transform(X_val))

print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))


print("SVC")
y_pred_train = svc.predict(scaler.transform(X_train))
y_pred_val = svc.predict(scaler.transform(X_val))

print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))


print("Ridge")
y_pred_train = ridge.predict(scaler.transform(X_train))
y_pred_val = ridge.predict(scaler.transform(X_val))

print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))

In [ ]:
X_train_extended = np.concatenate([X_train,
                                   knn.predict_proba(scaler.transform(X_train)),
                                   svc.predict_proba(scaler.transform(X_train)),
                                   ridge.predict(scaler.transform(X_train))[:, None],
                                   ], axis=1)

X_val_extended = np.concatenate([X_val,
                                   knn.predict_proba(scaler.transform(X_val)),
                                   svc.predict_proba(scaler.transform(X_val)),
                                   ridge.predict(scaler.transform(X_val))[:, None],
                                   ], axis=1)

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
cb_model = CatBoostClassifier(iterations=1200,
                        depth=6,
                        learning_rate=0.03,
                        loss_function='Logloss',
                        auto_class_weights='Balanced',
                        verbose=True)

cb_model.fit(X_train, y_train, eval_set=(X_val, y_val))
y_pred_train = cb_model.predict(X_train)
y_pred_val = cb_model.predict(X_val)

print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))

In [81]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=None, shuffle=False)

models = []
y = np.array(y)
for i, (train_index, test_index) in enumerate(kf.split(f)):
    X_train = f[train_index]
    y_train = y[train_index]
    X_val = f[test_index]
    y_val = y[test_index]

    cb_model = CatBoostClassifier(iterations=700,
                            depth=4,
                            learning_rate=0.03,
                            loss_function='Logloss',
                            auto_class_weights='Balanced',
                            verbose=False)

    cb_model.fit(X_train, y_train, eval_set=(X_val, y_val))
    y_pred_train = cb_model.predict(X_train)
    y_pred_val = cb_model.predict(X_val)

    print(classification_report(y_true=y_train, y_pred=y_pred_train))
    print(f1_score(y_true=y_train, y_pred=y_pred_train))

    print(classification_report(y_true=y_val, y_pred=y_pred_val))
    print(f1_score(y_true=y_val, y_pred=y_pred_val))
    models.append(cb_model)


              precision    recall  f1-score   support

           0       0.96      0.88      0.91      6687
           1       0.74      0.90      0.81      2665

    accuracy                           0.88      9352
   macro avg       0.85      0.89      0.86      9352
weighted avg       0.90      0.88      0.89      9352

0.814186322755812
              precision    recall  f1-score   support

           0       0.94      0.80      0.86      1642
           1       0.64      0.87      0.74       696

    accuracy                           0.82      2338
   macro avg       0.79      0.83      0.80      2338
weighted avg       0.85      0.82      0.82      2338

0.7406955460646736
              precision    recall  f1-score   support

           0       0.96      0.87      0.91      6759
           1       0.73      0.90      0.81      2593

    accuracy                           0.88      9352
   macro avg       0.84      0.89      0.86      9352
weighted avg       0.89      0.88    

In [ ]:
y_pred_train = cb_model.predict(X_train)
y_pred_val = cb_model.predict(X_val)


print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))

              precision    recall  f1-score   support

           0       0.95      0.88      0.91      6787
           1       0.74      0.87      0.80      2565

    accuracy                           0.88      9352
   macro avg       0.84      0.88      0.86      9352
weighted avg       0.89      0.88      0.88      9352

0.7970005356186395
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      1542
           1       0.77      0.82      0.80       796

    accuracy                           0.86      2338
   macro avg       0.84      0.85      0.84      2338
weighted avg       0.86      0.86      0.86      2338

0.7953573610262676


In [82]:
f1 = sents_decomp.transform(sents_vectorizer.transform(X_texts_test))
f2 = emb_decomp.transform(X_embeddings_test)
f3 = X_test_most_words
f4 = X_test_top_words
f5 = X_embeddings_test_cosine
f6 = X_embeddings_test_euclidean
# f7 = X_noise_proba_test_data[:, None]
# f8 = X_embeddings_min_dist_test
# f9 = X_embeddings_cosine_min_dist_test
f10 = X_tfidf_min_dist_test
f11 = X_tfidf_cosine_min_dist_test
f_test = np.concatenate([f1, f2, f3, f4, f5, f6, f10, f11], axis=1)
X_test = f_test
print(X_test.shape)

(16627, 193)


In [83]:
predicts = []

for model in models:
    predicts.append(model.predict(X_test))


predicts = np.vstack(predicts).T

In [ ]:
predicts.shape

(16627, 5)

In [ ]:
preds = np.sum(predicts, axis=1)

In [ ]:
idxs = np.logical_and(preds != 5, preds != 0)

In [ ]:
unique, count = np.unique(preds[idxs], return_counts=True)

In [ ]:
print(unique, count)

[1 2 3 4] [510 306 324 485]


In [84]:
final_predicts = ((np.sum(predicts, axis=1) / 5) > 0.5).astype(int)

In [85]:
X_test = f_test
df = pd.DataFrame({'pair_id': pairs, 'target': final_predicts})
df.to_csv("cb_ansemble_whole_fucking_model_м2.csv", index=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [98]:
X_test = f_test
df = pd.DataFrame({'pair_id': pairs, 'target': cb_model.predict(f_test)})
df.to_csv("cb_less_features.csv", index=None)